# AKI Seminar2 Demo
## Finetuning of LLaMA

by Syon Kadkade


Table of contents

> [Install packages](#install)   
> [Import libaries](#imports)   
> [Lorem Ipsum]()


--------------
<a id="install"></a>
### Install packages[Emoji]

**Description**:   
lorem ipsum

In [11]:
!pip install accelerate --quiet
!pip install bitsandbytes --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 2.5 MB/s eta 0:00:00


In [4]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.6 MB/s eta 0:00:00


In [15]:
#Maybe delete
!pip install -i https://test.pypi.org/simple/ bitsandbytes

Looking in indexes: https://test.pypi.org/simple/


In [13]:
#Maybe delete
!git clone https://github.com/timdettmers/bitsandbytes.git
!cd bitsandbytes

Cloning into 'bitsandbytes'...
remote: Enumerating objects: 5035, done.
remote: Counting objects: 100% (2855/2855), done.
remote: Compressing objects: 100% (513/513), done.
remote: Total 5035 (delta 2635), reused 2382 (delta 2340), pack-reused 2180
Receiving objects: 100% (5035/5035), 1.43 MiB | 5.33 MiB/s, done.
Resolving deltas: 100% (3495/3495), done.


---------------
<a id="imports"></a>
### Import libaries [Emoji]
**Description**:   
Load all necessary libaries.

In [2]:
import torch
import transformers
from accelerate import Accelerator
from transformers import LlamaForCausalLM, LlamaTokenizer

-----------------
### Load LLaMA-7B-Model[Emoji]

**Description**:  
lorem ipsum dolor sit amet

**Resources**:
- [Tutorial](#https://www.youtube.com/watch?v=t68IV5t5UOA)
- [Hugging Face: LLaMA-7B-Model](https://huggingface.co/docs/transformers/main/model_doc/llama)
- [Hugging Face: LLaMA weights](https://huggingface.co/luodian/llama-7b-hf)
- [Hugging Face: OpenLlama](#https://huggingface.co/openlm-research/open_llama_7b)

**Note**: I use a model that has the weights in it and we introduce these into the actual LLaMA model. Normally you have to request the weights from Meta AI by filling out a form. I have filled it out several times but there is no response from them.

In [6]:
tokenizer = LlamaTokenizer.from_pretrained('openlm-research/open_llama_7b')

ImportError: 
LlamaTokenizer requires the SentencePiece library but it was not found in your environment. Checkout the instructions on the
installation page of its repo: https://github.com/google/sentencepiece#installation and follow the ones
that match your environment. Please note that you may need to restart your runtime after installation.


In [ ]:
model = LlamaForCausalLM.from_pretrained("openlm-research/open_llama_7b")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/507 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]